In [70]:
import pandas as pd
import numpy as np
import random

def clean_separate_df(df, yr):
    all_players = df[df.year==yr][['player','pos','adp_by_pos','adp','ppr_pts','pos_rank','year']].sort_values('adp').reset_index(drop=True)
    qb = all_players[all_players.pos=='QB']
    rb = all_players[all_players.pos=='RB']
    wr = all_players[all_players.pos=='WR']
    te = all_players[all_players.pos=='TE']
    return all_players, qb, rb, wr, te


def create_strats(strat):
    if strat == 'h':
        return ['hero_rb', ['RB','WR','TE','QB','WR','WR','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'z':
        return ['zero_rb', ['WR','WR','TE','QB','VAL','VAL','RB','RB','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'r':
        return ['rb_heavy', ['RB','RB','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    else:
        return ['val',['VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    
def create_owners_strats(hero,zero,rb_heavy,value):
    strats = []
    for i in range(0,hero):
        strats.append(create_strats('hero'))
    for i in range(0,zero):
        strats.append(create_strats('zero'))
    for i in range(0,rb_heavy):
        strats.append(create_strats('rb_heavy'))
    for i in range(0,value):
        strats.append(create_strats('value'))
    return strats

def one_random_strat():
    '''h - hero, z - zero, r - rb_heavy, v - value'''
    return random.choice(['h','z','r','v','v','v'])

def create_league():
    league_draft_strat= []
    for i in range(0,12):
        league_draft_strat.append(one_random_strat())
        
    strats = []
    for l in league_draft_strat:
        strats.append(create_strats(l))
        
    return strats

class Cheatsheet:
    def __init__(self, all_players, qb, rb, wr, te):
        self.all_players = all_players
        self.qb = qb
        self.rb = rb
        self.wr = wr
        self.te = te
      
    def draft_qb(self,owner):
        selection = self.qb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.qb = self.qb[1:]
        owner.qb_count+=1
        return selection
        
    def draft_rb(self,owner):
        selection = self.rb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.rb = self.rb[1:]
        owner.rb_count+=1
        return selection
    
    def draft_wr(self,owner):
        selection = self.wr.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.wr = self.wr[1:]
        owner.wr_count+=1
        return selection
    
    def draft_te(self,owner):
        selection = self.te.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.te = self.te[1:]
        owner.te_count+=1
        return selection
    
    def draft_not_qb(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'QB':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_te(owner)
    
    def draft_not_te(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'TE':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_qb(owner)
        
    def draft_not_wr(self,owner):
        pass
    
    def draft_value(self,owner): 
        selection = self.all_players.iloc[0,:]
        if self.all_players.iloc[0,:].pos == 'QB':
            self.qb = self.qb.drop(index = selection.name)
            owner.qb_count+=1
        elif self.all_players.iloc[0,:].pos == 'RB':
            self.rb = self.rb.drop(index = selection.name)
            owner.rb_count+=1
        elif self.all_players.iloc[0,:].pos == 'WR':
            self.wr = self.wr.drop(index = selection.name)
            owner.wr_count+=1
        else:
            self.te = self.te.drop(index = selection.name)
            owner.te_count+=1
        self.all_players = self.all_players.iloc[1:]
        return selection
    

class Team_owner:
    def __init__(self, strat, draft_pos, draft_num, team_num):
        '''
        strat - draft strategy as tuple
        draft_pos - draft position 1-12
        draft_num - represents a unique league per season
        '''
        self.team = []
        self.strat_name = strat[0]      
        self.strat_order = strat[1]
        self.draft_pos = draft_pos
        self.draft_num = draft_num
        self.team_num = team_num
        self.qb_count = 0
        self.te_count = 0
        self.wr_count = 0
        self.rb_count = 0
        
        
    def draft_player(self,cheatsheet):
        if len(self.team) == 13 and self.te_count == 0: #Draft a TE in 10th round if no TEs
            self.strat_order.pop(0)
            return cheatsheet.draft_te(self)
        
        elif len(self.team) == 8 and self.qb_count == 0: #Draft a QB in 9th round if no QBs
            self.strat_order.pop(0)
            return cheatsheet.draft_qb(self)
        
        elif len(self.team) == 7 and self.rb_count == 1: #Draft a RB in 8th round if only 1 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 1: #Draft a WR in 8th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif len(self.team) == 6 and self.rb_count == 0: #Draft a RB in 7th round if only 0 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 0: #Draft a WR in 7th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif self.qb_count == 2 and self.strat_order[0]=='VAL': #Don't draft more than 2 QB
            self.strat_order.pop(0)
            return cheatsheet.draft_not_qb(self)
        
        elif self.te_count == 2 and self.strat_order[0]=='VAL': # Don't draft more than 2 TE
            self.strat_order.pop(0)
            return cheatsheet.draft_not_te(self)
        
        
#         elif self.rb_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_wr(self)
        
#         elif self.wr_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_rb(self)
        
        else:
            pos = self.strat_order.pop(0)
            if pos == 'QB':
                self.qb_count+=1
                return cheatsheet.draft_qb(self)
            elif pos == 'RB':
                return cheatsheet.draft_rb(self)
            elif pos == 'WR':
                return cheatsheet.draft_wr(self)
            elif pos == 'TE':
                return cheatsheet.draft_te(self)
            else:
                return cheatsheet.draft_value(self)

def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    df = df.reset_index(drop=True)
    df.index+=1
    df['pick_number'] = df.index
    
    top7 = [] 
    top7.append(df[df.pos=='QB'].sort_values('ppr_pts',ascending=False).head(1))
    top7.append(df[df.pos=='RB'].sort_values('ppr_pts',ascending=False).head(2))
    df = df.drop(df[df.pos=='RB'].sort_values('ppr_pts',ascending=False).head(2).index)
    top7.append(df[df.pos=='WR'].sort_values('ppr_pts',ascending=False).head(2))
    df = df.drop(df[df.pos=='WR'].sort_values('ppr_pts',ascending=False).head(2).index)
    top7.append(df[(df.pos=='RB')|(df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(1))
    top7.append(df[df.pos=='TE'].sort_values('ppr_pts',ascending=False).head(1))
    df = pd.concat(top7)
    df['total_pts'] = df.ppr_pts.sum()
    return df



            
    

# Create 100 mock leagues each year to run analysis on

In [71]:
df = pd.read_csv('season1.csv',index_col=0)
owners_teams = [] 
count = 0
for season in range(2016,2023):
    for sim in range(0,100):
        all_players, qb, rb, wr, te = clean_separate_df(df,season)
        strats = create_league()
        cheat = Cheatsheet(all_players, qb, rb, wr, te)

        owners = [] #Create 12 team owners with 'value_strat' drafting strategy
        for i, s in zip(range(0,12),strats.copy()):
            owners.append(Team_owner(s,i+1,sim,count))
            count+=1

        owners.reverse()
        for i in range(0,14): #Draft 14 rounds for each team owner
            owners.reverse()
            for o in owners:
                o.team.append(o.draft_player(cheat))

        owners_teams.append(owners)

all_teams = []
for t in owners_teams:
    for o in t:
        df = pd.concat(o.team,axis=1).T
        df['draft_pos'] = o.draft_pos
        df['draft_strat'] = o.strat_name
        df['league_id'] = o.draft_num
        df['team_num'] = o.team_num
        all_teams.append(df) 

df = pd.concat(all_teams)
    
   

In [72]:
df[df.team_num==1]

,player,pos,adp_by_pos,adp,ppr_pts,pos_rank,year,draft_pos,draft_strat,league_id,team_num
2,David Johnson,RB,1,4.0,407.8,1,2016,2,rb_heavy,0,1
23,Mark Ingram,RB,10,26.0,242.2,8,2016,2,rb_heavy,0,1
25,Sammy Watkins,WR,15,28.0,83.0,91,2016,2,rb_heavy,0,1
46,Doug Baldwin,WR,25,50.0,253.6,8,2016,2,rb_heavy,0,1
49,Russell Wilson,QB,3,54.0,270.1,10,2016,2,rb_heavy,0,1
70,Frank Gore,RB,26,76.0,214.2,12,2016,2,rb_heavy,0,1
73,Matt Jones,RB,28,79.0,75.3,59,2016,2,rb_heavy,0,1
91,Philip Rivers,QB,11,98.0,258.9,14,2016,2,rb_heavy,0,1
94,Devin Funchess,WR,40,101.0,86.1,89,2016,2,rb_heavy,0,1
119,Spencer Ware,RB,43,131.0,193.8,17,2016,2,rb_heavy,0,1


In [73]:
com = []
for seas in range(2016,2023): # for 'year'
    for num in range(0,100): #for 'draft_num'
        for i in range(1,13): #for 'draft_pos'
            temp = df[(df.year==seas)&(df.league_id==num)&(df.draft_pos==i)]
            com.append(top7(temp))
top = pd.concat(com) #df of each team owners top 7 starters with 'total_pts' column

In [75]:
top.to_csv('top7.csv')
df.to_csv('sim100.csv')

In [76]:
top.total_pts.mean()

1532.5970339127139

In [41]:
top.groupby('draft_strat')['total_pts'].mean().sort_values()

draft_strat
rb_heavy    1526.276919
val         1530.627839
zero_rb     1532.944549
hero_rb     1540.787168
Name: total_pts, dtype: float64

In [54]:
top.groupby('draft_pos')['total_pts'].mean().sort_values(ascending=False)

draft_pos
3     1576.332143
4     1570.606286
5     1547.696550
6     1546.672571
9     1544.605103
1     1535.379400
10    1533.004143
2     1526.262571
7     1520.332286
8     1519.305571
11    1497.239571
12    1465.491857
Name: total_pts, dtype: float64

In [67]:
top[(top.draft_pos==3)&(top.draft_strat=='hero_rb')&(top.year==2019)].iloc[7:14,:]

,player,pos,adp,ppr_pts,pos_rank,year,draft_pos,draft_strat,league_id,team_num,pick_number,total_pts
4,Matt Ryan,QB,70.0,267.3,11,2019,3,hero_rb,3,3638,4,1741.8
1,Christian McCaffrey,RB,3.0,471.2,1,2019,3,hero_rb,3,3638,1,1741.8
7,Latavius Murray,RB,79.0,157.2,28,2019,3,hero_rb,3,3638,7,1741.8
5,Kenny Golladay,WR,48.0,248.0,9,2019,3,hero_rb,3,3638,5,1741.8
11,Courtland Sutton,WR,132.0,222.4,19,2019,3,hero_rb,3,3638,11,1741.8
12,Carlos Hyde,RB,160.0,153.2,30,2019,3,hero_rb,3,3638,12,1741.8
3,George Kittle,TE,28.0,222.5,2,2019,3,hero_rb,3,3638,3,1741.8


In [45]:
top.groupby(['draft_pos','draft_strat'])[['total_pts']].mean().sort_values(['draft_pos','total_pts'])

total_pts
draft_pos draft_strat             
1         val          1512.764368
          rb_heavy     1528.944506
          zero_rb      1564.271186
          hero_rb      1585.751429
2         val          1512.066667
          rb_heavy     1512.792241
          zero_rb      1544.266923
          hero_rb      1560.091525
3         zero_rb      1553.078846
          rb_heavy     1569.379832
          val          1575.352355
          hero_rb      1607.361207
4         val          1562.517355
          zero_rb      1565.436190
          rb_heavy     1575.752985
          hero_rb      1599.070408
5         zero_rb      1514.139837
          val          1547.096077
          hero_rb      1564.441818
          rb_heavy     1570.421818
6         zero_rb      1514.483333
          hero_rb      1531.239683
          rb_heavy     1558.957983
          val          1561.321362
7         rb_heavy     1492.828926
          val          1516.855193
          zero_rb      1533.214504
          hero_rb      1545.666667
8         rb_heavy     1504.172727
          zero_rb      1514.676230
          val          1523.822865
          hero_rb      1524.920952
9         rb_heavy     1533.294253
          hero_rb      1536.206780
          val          1545.188102
          zero_rb      1562.129915
10        hero_rb      1498.689623
          rb_heavy     1533.971212
          val          1540.530110
          zero_rb      1540.857000
11        rb_heavy     1461.544545
          hero_rb      1491.248739
          val          1506.628610
          zero_rb      1508.867010
12        rb_heavy     1459.450893
          hero_rb      1461.671774
          val          1466.060218
          zero_rb      1475.200000

In [46]:
top.groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
3,hero_rb,1607.361207,812
4,hero_rb,1599.070408,686
1,hero_rb,1585.751429,735
4,rb_heavy,1575.752985,938
3,val,1575.352355,2527
5,rb_heavy,1570.421818,770
3,rb_heavy,1569.379832,833
4,zero_rb,1565.436190,735
5,hero_rb,1564.441818,770


In [47]:
top[top.draft_strat=='hero_rb'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count','max','min']).\
sort_values('mean',ascending=False)

,,mean,count,max,min
draft_pos,draft_strat,,,,
3,hero_rb,1607.361207,812,1971.0,1166.2
4,hero_rb,1599.070408,686,2149.9,1111.9
1,hero_rb,1585.751429,735,2008.2,1306.4
5,hero_rb,1564.441818,770,2013.0,1156.0
2,hero_rb,1560.091525,826,1971.0,1019.7
7,hero_rb,1545.666667,777,2034.2,1039.6
9,hero_rb,1536.206780,826,1861.5,1134.3
6,hero_rb,1531.239683,882,2018.7,1138.1
8,hero_rb,1524.920952,735,2028.1,1085.0


In [48]:
top[top.draft_strat=='zero_rb'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
4,zero_rb,1565.436190,735
1,zero_rb,1564.271186,826
9,zero_rb,1562.129915,819
3,zero_rb,1553.078846,728
2,zero_rb,1544.266923,910
10,zero_rb,1540.857000,700
7,zero_rb,1533.214504,917
8,zero_rb,1514.676230,854
6,zero_rb,1514.483333,924


In [49]:
top[top.draft_strat=='val'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
3,val,1575.352355,2527
4,val,1562.517355,2541
6,val,1561.321362,2261
5,val,1547.096077,2498
9,val,1545.188102,2471
10,val,1540.530110,2534
8,val,1523.822865,2541
7,val,1516.855193,2359
1,val,1512.764368,2436


In [50]:
top[top.draft_strat=='rb_heavy'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
4,rb_heavy,1575.752985,938
5,rb_heavy,1570.421818,770
3,rb_heavy,1569.379832,833
6,rb_heavy,1558.957983,833
10,rb_heavy,1533.971212,924
9,rb_heavy,1533.294253,783
1,rb_heavy,1528.944506,901
2,rb_heavy,1512.792241,812
8,rb_heavy,1504.172727,770


In [51]:
top.head()

,player,pos,adp,ppr_pts,pos_rank,year,draft_pos,draft_strat,league_id,team_num,pick_number,total_pts
9,Derek Carr,QB,107.0,268.5,11,2016,1,val,0,0,9,1517.3
2,Mark Ingram,RB,26.0,242.2,8,2016,1,val,0,0,2,1517.3
10,Spencer Ware,RB,131.0,193.8,17,2016,1,val,0,0,10,1517.3
1,Antonio Brown,WR,1.0,307.3,1,2016,1,val,0,0,1,1517.3
3,Jarvis Landry,WR,27.0,231.3,14,2016,1,val,0,0,3,1517.3


In [52]:
top.groupby(['pick_number','pos'])['total_pts'].agg(['mean','count'])

mean  count
pick_number pos                    
1           RB   1557.829553   4612
            TE   1608.340659     91
            WR   1518.270261   2495
2           QB   1563.635743    249
            RB   1542.785770   2579
            TE   1567.982227    467
            WR   1546.825639   3171
3           QB   1535.891818    330
            RB   1514.962598   1532
            TE   1551.994394   2212
            WR   1541.663804   1898
4           QB   1539.330587   2877
            RB   1519.841620   1420
            TE   1534.140909     22
            WR   1538.528054   2374
5           QB   1480.555645    124
            RB   1506.772081   1730
            TE   1522.740870    115
            WR   1545.425830   3101
6           QB   1531.021964    560
            RB   1524.962837   1410
            TE   1546.451907    472
            WR   1520.682102   1732
7           QB   1496.113298    564
            RB   1539.937062   1600
            TE   1501.905184    463
            WR   1567.456293   1025
8           QB   1539.290717    614
            RB   1539.578304   1627
            TE   1560.637052    502
            WR   1511.712171   1594
9           QB   1520.439464   1680
            RB   1503.767381    653
            TE   1488.725225    333
            WR   1554.976633   1301
10          QB   1574.016203    395
            RB   1484.692377    774
            TE   1500.973256    430
            WR   1513.144377    987
11          QB   1494.386406    434
            RB   1528.891827    416
            TE   1526.845042    353
            WR   1519.137489   1139
12          QB   1543.276471    323
            RB   1458.240230    522
            TE   1552.957648    961
            WR   1509.253659    574
13          QB   1555.337313    201
            RB   1494.794882    762
            TE   1582.967173    527
            WR   1480.095140    535
14          QB   1580.536735     49
            RB   1632.892562    121
            TE   1468.274793   1452
            WR   1544.758013    312

### What other questions can i ask using these simulations? What if I try it just for the past 3 years?

How can i use this info the upcoming ffb season?

Can have s